In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`org.apache.spark::spark-hive:2.4.5`

import $ivy.$                                  

import $ivy.$                                   

In [2]:
import sys.process._

import org.apache.log4j._
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.SparkFiles
import org.apache.spark.sql._
// import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._


import sys.process._


import org.apache.log4j._

import org.apache.spark.SparkFiles

import org.apache.spark.sql._
// import org.apache.spark.sql.SQLContext

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._


In [3]:
val spark = (
    AmmoniteSparkSession
    .builder()
    .master("local[*]")
    .enableHiveSupport()
    .getOrCreate()
)

import spark.implicits._
spark.sparkContext.setLogLevel("FATAL")

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@360f5102
import spark.implicits._


In [ ]:
// spark.catalog.clearCache()
// spark.close()
// spark.stop()

<br>**Загрузим источник**

In [4]:
spark.sparkContext.addFile("https://www.sba.gov/sites/default/files/data.json")

val data = spark.read.json("file://"+SparkFiles.get("data.json"))
val dataset_meta = data.select("@context","@type","conformsTo","describedBy").toJSON.collect()(0)

data.printSchema

root
 |-- @context: string (nullable = true)
 |-- @type: string (nullable = true)
 |-- conformsTo: string (nullable = true)
 |-- dataset: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- @type: string (nullable = true)
 |    |    |-- accessLevel: string (nullable = true)
 |    |    |-- accrualPeriodicity: string (nullable = true)
 |    |    |-- bureauCode: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- contactPoint: struct (nullable = true)
 |    |    |    |-- @type: string (nullable = true)
 |    |    |    |-- fn: string (nullable = true)
 |    |    |    |-- hasEmail: string (nullable = true)
 |    |    |-- dataQuality: boolean (nullable = true)
 |    |    |-- describedBy: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- distribution: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- @type: string (

data: DataFrame = [@context: string, @type: string ... 3 more fields]
dataset_meta: String = "{\"@context\":\"https://project-open-data.cio.gov/v1.1/schema/catalog.jsonld\",\"@type\":\"dcat:Catalog\",\"conformsTo\":\"https://project-open-data.cio.gov/v1.1/schema\",\"describedBy\":\"https://project-open-data.cio.gov/v1.1/schema/catalog.json\"}"

<br>**Сохраним источник**
<br>**в реальной ситуации будем партиционировать по дате загрузки**
<br>**Или по дате модификации записи ```modified```, но это усложнит процесс апдейта данных**
<br>**На данный момент сложно сразу сказать как будет оптимальнее**

In [5]:
(
    data
    //.withColumn("load_date", expr("CURRENT_DATE"))
    .write
    .mode("overwrite")
    //.partitionBy("load_date")
    .json("hdfs/source_daily_batch_json")
)

<br>**Будем считать, что схема прогруженной части данных обладает полным набором полей**
<br>**Сохраним для дальнейшего использования**

In [ ]:
// import java.io._

// val schemaDDL1 = data.schema.toDDL
// // spark.sparkContext.parallelize(Array(schemaDDL)).toDF.toJSON.coalesce(1).write.mode("overwrite").json("hdfs/meta_schemaDDL")
// val writerDDL1 = new PrintWriter(new File("schemaDDL1.txt"))
// writerDDL1.write(schemaDDL1)
// writerDDL1.close()

// import java.io._
// val schemaDDL2 = data.select(explode($"dataset") as "dataset").select("dataset.*").schema.toDDL
// val writerDDL2 = new PrintWriter(new File("schemaDDL2.txt"))
// writerDDL2.write(schemaDDL2)
// writerDDL2.close()

// val schemaJSON = data.schema.json
// // spark.sparkContext.parallelize(Array(schemaJSON)).toDF.toJSON.coalesce(1).write.mode("overwrite").json("hdfs/meta_schemaJSON")
// val writerJSON = new PrintWriter(new File("schemaJSON.json"))
// writerJSON.write(schemaJSON)
// writerJSON.close()

In [6]:
val schemaDDL1str = "`@context` STRING,`@type` STRING,`conformsTo` STRING,`dataset` ARRAY<STRUCT<`@type`: STRING, `accessLevel`: STRING, `accrualPeriodicity`: STRING, `bureauCode`: ARRAY<STRING>, `contactPoint`: STRUCT<`@type`: STRING, `fn`: STRING, `hasEmail`: STRING>, `dataQuality`: BOOLEAN, `describedBy`: STRING, `description`: STRING, `distribution`: ARRAY<STRUCT<`@type`: STRING, `accessURL`: STRING, `describedBy`: STRING, `description`: STRING, `downloadURL`: STRING, `format`: STRING, `mediaType`: STRING, `title`: STRING>>, `identifier`: STRING, `isPartOf`: STRING, `issued`: STRING, `keyword`: ARRAY<STRING>, `landingPage`: STRING, `language`: ARRAY<STRING>, `license`: STRING, `modified`: STRING, `programCode`: ARRAY<STRING>, `publisher`: STRUCT<`@type`: STRING, `name`: STRING>, `rights`: STRING, `theme`: ARRAY<STRING>, `title`: STRING>>,`describedBy` STRING"
val schemaDDL2str = "`@type` STRING,`accessLevel` STRING,`accrualPeriodicity` STRING,`bureauCode` ARRAY<STRING>,`contactPoint` STRUCT<`@type`: STRING, `fn`: STRING, `hasEmail`: STRING>,`dataQuality` BOOLEAN,`describedBy` STRING,`description` STRING,`distribution` ARRAY<STRUCT<`@type`: STRING, `accessURL`: STRING, `describedBy`: STRING, `description`: STRING, `downloadURL`: STRING, `format`: STRING, `mediaType`: STRING, `title`: STRING>>,`identifier` STRING,`isPartOf` STRING,`issued` STRING,`keyword` ARRAY<STRING>,`landingPage` STRING,`language` ARRAY<STRING>,`license` STRING,`modified` STRING,`programCode` ARRAY<STRING>,`publisher` STRUCT<`@type`: STRING, `name`: STRING>,`rights` STRING,`theme` ARRAY<STRING>,`title` STRING"

schemaDDL1str: String = "`@context` STRING,`@type` STRING,`conformsTo` STRING,`dataset` ARRAY<STRUCT<`@type`: STRING, `accessLevel`: STRING, `accrualPeriodicity`: STRING, `bureauCode`: ARRAY<STRING>, `contactPoint`: STRUCT<`@type`: STRING, `fn`: STRING, `hasEmail`: STRING>, `dataQuality`: BOOLEAN, `describedBy`: STRING, `description`: STRING, `distribution`: ARRAY<STRUCT<`@type`: STRING, `accessURL`: STRING, `describedBy`: STRING, `description`: STRING, `downloadURL`: STRING, `format`: STRING, `mediaType`: STRING, `title`: STRING>>, `identifier`: STRING, `isPartOf`: STRING, `issued`: STRING, `keyword`: ARRAY<STRING>, `landingPage`: STRING, `language`: ARRAY<STRING>, `license`: STRING, `modified`: STRING, `programCode`: ARRAY<STRING>, `publisher`: STRUCT<`@type`: STRING, `name`: STRING>, `rights`: STRING, `theme`: ARRAY<STRING>, `title`: STRING>>,`describedBy` STRING"
schemaDDL2str: String = "`@type` STRING,`accessLevel` STRING,`accrualPeriodicity` STRING,`bureauCode` ARRAY<STRING>,`con

In [7]:
val ddlSchema1 = StructType.fromDDL(schemaDDL1str)
// ddlSchema1.printTreeString()
val ddlSchema2 = StructType.fromDDL(schemaDDL2str)
// ddlSchema2.printTreeString()

ddlSchema1: StructType = StructType(
  StructField("@context", StringType, true, {}),
  StructField("@type", StringType, true, {}),
  StructField("conformsTo", StringType, true, {}),
  StructField(
    "dataset",
    ArrayType(
      StructType(
        StructField("@type", StringType, true, {}),
        StructField("accessLevel", StringType, true, {}),
        StructField("accrualPeriodicity", StringType, true, {}),
        StructField("bureauCode", ArrayType(StringType, true), true, {}),
        StructField(
          "contactPoint",
          StructType(
            StructField("@type", StringType, true, {}),
            StructField("fn", StringType, true, {}),
            StructField("hasEmail", StringType, true, {})
          ),
          true,
          {}
        ),
        StructField("dataQuality", BooleanType, true, {}),
        StructField("describedBy", StringType, true, {}),
        StructField("description", StringType, true, {}),
        StructField(
          "distribut

<br>**Если дата-аналитики не против работать с комплексными структурами - сохраним для них часть данных в Hive таблицу, партицированную по дате загрузки**
<br>**Для батча - сохраним часть данных в parquet**
<br>**Что делать с дублямя-как апдейтить и как учесть дату модификации - не решил**

In [8]:
spark.sql(""" SET hive.exec.dynamic.partition=true """)
spark.sql(""" SET hive.exec.dynamic.partition.mode=nonstrict """)

spark.sql("""
DROP TABLE IF EXISTS DATASET_DAILY_FULL
""")

spark.sql("""
CREATE TABLE DATASET_DAILY_FULL (
    `type` STRING,
    `accessLevel` STRING,
    `accrualPeriodicity` STRING,
    `bureauCode` ARRAY<STRING>,
    `contactPoint` STRUCT<`type`: STRING, `fn`: STRING, `hasEmail`: STRING>,
    `dataQuality` BOOLEAN,
    `describedBy` STRING,
    `description` STRING,
    `distribution` ARRAY<STRUCT<`type`: STRING, `accessURL`: STRING, `describedBy`: STRING, `description`: STRING, `downloadURL`: STRING, `format`: STRING, `mediaType`: STRING, `title`: STRING>>,
    `identifier` STRING,
    `isPartOf` STRING,
    `issued` STRING,
    `keyword` ARRAY<STRING>,
    `landingPage` STRING,
    `language` ARRAY<STRING>,
    `license` STRING,
    `modified` STRING,
    `programCode` ARRAY<STRING>,
    `publisher` STRUCT<`type`: STRING, `name`: STRING>,
    `rights`  STRING,
    `theme` ARRAY<STRING>,
    `title` STRING
)
PARTITIONED BY (load_date date)
STORED AS PARQUET 
""")

spark.sql("""
DROP TABLE IF EXISTS DATASET_DAILY
""")


res7_0: DataFrame = [key: string, value: string]
res7_1: DataFrame = [key: string, value: string]
res7_2: DataFrame = []
res7_3: DataFrame = []
res7_4: DataFrame = []

<br>**Используя ранее сохранные данные ```data.json``` и схему ```ddlSchema1``` сэмулируем запись в табилицу и hdfs через частичное сэмплирование, итерируясь по ```dateRange```**

In [9]:
val dateRange: Array[String] = spark.sql("SELECT explode(sequence(to_date('2020-01-01'), to_date('2020-01-10'), interval 1 day))").collect.map(_(0).toString)

dateRange: Array[String] = Array(
  "2020-01-01",
  "2020-01-02",
  "2020-01-03",
  "2020-01-04",
  "2020-01-05",
  "2020-01-06",
  "2020-01-07",
  "2020-01-08",
  "2020-01-09",
  "2020-01-10"
)

In [14]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

def uploadData(spark: org.apache.spark.sql.SparkSession, current_date: String) = {
    val records = (
        spark.read.schema(ddlSchema1).json("hdfs/source_daily_batch_json")
        .select(explode($"dataset") as "dataset")
        .select("dataset.*")
        .sample(false, 0.66)
        .withColumn("load_date", expr(s"cast('${current_date}' as date)"))
        )
    val cnt1 = records.select("identifier").distinct.count
    
    // данные для батч-джобы
    records.write.mode("overwrite").partitionBy("load_date").parquet("hdfs/source_daily_batch_parquet")
    
    // данные для аналитиков
    records.createOrReplaceTempView("RECORDS")
    spark.sql("""
        INSERT OVERWRITE TABLE 
            DATASET_DAILY_FULL PARTITION(load_date) 
        SELECT 
            *
        FROM
            RECORDS""")
    val cnt2 = spark.sql(s""" SELECT DISTINCT identifier FROM DATASET_DAILY_FULL WHERE load_date='${current_date}' """).count()
    
    //check 
    println("(uniq. identifier) "+current_date+": "+cnt1+", "+cnt2+" loaded from source")
}

defined function uploadData

In [15]:
uploadData(spark, dateRange(0))

(uniq. identifier) 2020-01-01: 621, 621 loaded from source


In [16]:
def sparkJob(spark: org.apache.spark.sql.SparkSession, schemaRec: types.StructType, current_date: String) = {
    
    // кол-во экземпляров по accessLevel, bureauCode, programCode и год из поля identifier
    // println(current_date)
    
    val records_full = spark.read.schema(ddlSchema2).parquet("hdfs/source_daily_batch_parquet/")
    val records_prev = records_full.filter(s" load_date < cast('${current_date}' as date) ").select($"identifier" as "identifier_prev").distinct
    val records_curr = records_full.filter(s" load_date = cast('${current_date}' as date) ")
    val records = records_prev.join(records_curr, expr("identifier_prev = identifier"), "outer")
    
    val dublrec = records.filter("identifier_prev is not null and identifier is not null").count
    println(s"REPORT: '${current_date}' \n\nfound dubl: " + dublrec+"\n")
    
    val results = records.filter("identifier_prev IS NULL").drop("identifier_prev")
    
    // TODO: replace DISTINCT by LAST(modified) ???
    // TODO: add check for empty batch
    
    val accessLevel = results.select($"identifier", $"accessLevel").distinct.groupBy("accessLevel").agg(count($"identifier")).show()
    val bureauCode  = results.select($"identifier", explode($"bureauCode") as "bureauCode").distinct.groupBy("bureauCode").agg(count($"identifier")).show()
    val programCode  = results.select($"identifier", explode($"programCode") as "programCode").distinct.groupBy("programCode").agg(count($"identifier")).show()
    val identifierYear = (
        results
        .select($"identifier", regexp_extract($"identifier", "(\\d{4})(-\\d{2}-\\d{3})", 1) as "identifierYear")
        .distinct.groupBy("identifierYear").agg(count($"identifier")).orderBy("identifierYear").show())
}

defined function sparkJob

<br> ```RUN EXPERIMENT```

In [17]:
for (current_date <- dateRange.slice(1, dateRange.length)) {
    
    uploadData(spark, current_date)
    sparkJob(spark, ddlSchema2, current_date)

}


(uniq. identifier) 2020-01-02: 596, 596 loaded from source
REPORT: '2020-01-02' 

found dubl: 393

+-----------+-----------------+
|accessLevel|count(identifier)|
+-----------+-----------------+
|     public|              196|
| non-public|                7|
+-----------+-----------------+

+----------+-----------------+
|bureauCode|count(identifier)|
+----------+-----------------+
|    028:00|              203|
+----------+-----------------+

+-----------+-----------------+
|programCode|count(identifier)|
+-----------+-----------------+
|    028:017|                6|
|    028:000|                8|
|    028:001|               37|
|    028:031|              150|
|    028:002|                2|
+-----------+-----------------+

+--------------+-----------------+
|identifierYear|count(identifier)|
+--------------+-----------------+
|          2013|                1|
|          2015|               44|
|          2016|               58|
|          2017|               18|
|          2018|  

In [20]:
spark.sql("""SHOW SCHEMAS""").show()
spark.sql("""SHOW TABLES IN DEFAULT""").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+

+--------+------------------+-----------+
|database|         tableName|isTemporary|
+--------+------------------+-----------+
| default|dataset_daily_full|      false|
|        |           records|       true|
+--------+------------------+-----------+



In [22]:
spark.sql("SELECT load_date, identifier FROM dataset_daily_full").show(3)

+----------+--------------------+
| load_date|          identifier|
+----------+--------------------+
|2020-01-07|SBA-OCIO-2015-08-002|
|2020-01-07|SBA-OCIO-2015-08-001|
|2020-01-07| SBA-OCA-2017-11-001|
+----------+--------------------+
only showing top 3 rows



In [23]:
spark.sql("SELECT load_date, count(identifier) as cnt, count(distinct identifier) uniq FROM dataset_daily_full group by load_date order by load_date").show()

+----------+---+----+
| load_date|cnt|uniq|
+----------+---+----+
|2020-01-01|621| 621|
|2020-01-02|596| 596|
|2020-01-03|604| 604|
|2020-01-04|576| 576|
|2020-01-05|577| 577|
|2020-01-06|618| 618|
|2020-01-07|604| 604|
|2020-01-08|606| 606|
|2020-01-09|617| 617|
|2020-01-10|624| 624|
+----------+---+----+



In [24]:
spark.sql("SELECT count(identifier) as cnt, count(distinct identifier) uniq FROM dataset_daily_full").show()

+----+----+
| cnt|uniq|
+----+----+
|6043| 922|
+----+----+



<br>**Если аналитика на уровне Спарка - не вариант, а комплексные типы - вызывут сложности, то можно вынести данные поля в отдельны сущности**

In [25]:
val records = (
    spark.read.schema(ddlSchema1).json("hdfs/source_daily_batch_json")
    .select(explode($"dataset") as "dataset")
    .select("dataset.*")
    .sample(false, 0.3)
)

records: Dataset[Row] = [@type: string, accessLevel: string ... 20 more fields]

In [26]:
println("TABLE EXAMPLE: "+"bureauCode".toUpperCase)
records.select($"identifier", $"modified", explode($"bureauCode") as "bureauCode").show(3)

println("TABLE EXAMPLE: "+"contactPoint".toUpperCase)
records.select($"identifier", $"modified", $"contactPoint.*").show(3)

println("TABLE EXAMPLE: "+"distribution".toUpperCase)
records.select($"identifier", $"modified", explode($"distribution") as "distribution").select($"identifier", $"distribution.*").show(3)

println("TABLE EXAMPLE: "+"keyword".toUpperCase)
records.select($"identifier", $"modified", explode($"keyword") as "keyword").show(3)

println("TABLE EXAMPLE: "+"programCode".toUpperCase)
records.select($"identifier", $"modified", explode($"programCode") as "programCode").show(3)

println("TABLE EXAMPLE: "+"publisher".toUpperCase)
records.select($"identifier", $"modified", $"publisher.*").show(3)

println("TABLE EXAMPLE: "+"theme".toUpperCase)
records.select($"identifier", $"modified", explode($"theme") as "theme").show(3)

TABLE EXAMPLE: BUREAUCODE
+--------------------+----------+----------+
|          identifier|  modified|bureauCode|
+--------------------+----------+----------+
|SBA-OCIO-2015-08-002|2019-11-30|    028:00|
|SBA-OCIO-2015-08-001|2020-06-12|    028:00|
| SBA-OCA-2017-11-004|2017-06-30|    028:00|
+--------------------+----------+----------+
only showing top 3 rows

TABLE EXAMPLE: CONTACTPOINT
+--------------------+----------+-------------+-------------+--------------------+
|          identifier|  modified|        @type|           fn|            hasEmail|
+--------------------+----------+-------------+-------------+--------------------+
|SBA-OCIO-2015-08-002|2019-11-30|vcard:Contact| Melvin Brown|mailto:melvin.bro...|
|SBA-OCIO-2015-08-001|2020-06-12|vcard:Contact| Melvin Brown|mailto:melvin.bro...|
| SBA-OCA-2017-11-004|2017-06-30|vcard:Contact|Ronald Whalen|mailto:Ronald.Wha...|
+--------------------+----------+-------------+-------------+--------------------+
only showing top 3 rows



In [27]:
spark.catalog.clearCache()
spark.close()
spark.stop()

<br> **Notes**

In [ ]:
// val cols = Array("type","accessURL","describedBy", "description", "downloadURL", "format","mediaType", "title").map(x=>count(col(x)))
// recordReaded.select($"identifier", explode($"distribution") as "distribution").select($"identifier", $"distribution.*").groupBy().agg(cols.head, cols.tail: _*).show

In [ ]:
// val cols = Array("@type","accessURL","describedBy", "description", "downloadURL", "format","mediaType", "title").map(x=>count(col(x)))
// records.select($"identifier", explode($"distribution") as "distribution").select($"identifier", $"distribution.*").groupBy().agg(cols.head, cols.tail: _*).show

In [ ]:
// val identifierDate = regexp_extract($"identifier", "(\\d{4}-\\d{2}-\\d{3})", 0)
// records.select("identifier").withColumn("identifierDate", identifierDate).write.mode("overwrite").partitionBy("identifierDate").parquet("data/stg_batch")

In [ ]:
// val identifierYear = regexp_extract($"identifier", "(\\d{4})(-\\d{2}-\\d{3})", 1)
// records.select("identifier").withColumn("identifierYear", identifierYear).write.mode("overwrite").partitionBy("identifierYear").parquet("data/stg_batch")